In [ ]:
import os
import sys
from datetime import datetime
import tempfile
import time
import json
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from codecarbon import EmissionsTracker
from vllm import LLM, SamplingParams
import pandas as pd


[codecarbon INFO @ 13:17:26] Energy consumed for RAM : 0.137686 kWh. RAM Power : 188.8243260383606 W
[codecarbon INFO @ 13:17:26] Energy consumed for all CPUs : 0.082174 kWh. Total CPU Power : 112.5 W
[codecarbon INFO @ 13:17:26] Energy consumed for all GPUs : 0.474365 kWh. Total GPU Power : 720.6452775384009 W
[codecarbon INFO @ 13:17:26] 0.694225 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:17:27] Energy consumed for RAM : 0.137737 kWh. RAM Power : 188.8243260383606 W
[codecarbon INFO @ 13:17:27] Energy consumed for all CPUs : 0.082204 kWh. Total CPU Power : 112.5 W
[codecarbon INFO @ 13:17:27] Energy consumed for all GPUs : 0.474496 kWh. Total GPU Power : 480.30496631462194 W
[codecarbon INFO @ 13:17:27] 0.694438 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:17:28] Energy consumed for RAM : 0.137789 kWh. RAM Power : 188.8243260383606 W
[codecarbon INFO @ 13:17:28] Energy consumed for all CPUs : 0.082235 kWh. Total CPU Power : 112.5 W
[codec

In [3]:
def text_gen_experiment(model_name, prompts, max_input_tokens=512, max_output_tokens=50, batch_size=8):
    """
    Wrapper for text generation experiments.
    Calls the generic experiment runner with the text generation inference function.
    """
    return run_experiment(model_name=model_name, 
                          prompts=prompts, 
                          inference_fn=run_gen_inference_with_metrics, 
                          task_type="text_generation",
                          max_input_tokens=max_input_tokens,
                          max_output_tokens=max_output_tokens,
                          batch_size=batch_size)

In [ ]:
# usage 
model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0"

ds = load_dataset("lighteval/pile_helm", "arxiv")["test"]
ds = ds.select(range(10))
prompts = [sample["text"] for sample in ds]

# Currently only works with pyrorch (vLLM requires more server space) - Choose backend: "pytorch", "vllm", "onnx", or "tensorrt"
backend = "pytorch"

use_optimum = False

experiment_results = run_experiment(
    model_name=model_name,
    prompts=prompts,
    inference_fn=run_gen_inference_with_metrics,
    task_type="text_generation",
    backend=backend,
    use_optimum=use_optimum,
    max_input_tokens=512,
    max_output_tokens=50,
    batch_size=8
)

[codecarbon INFO @ 12:32:32] [setup] RAM Tracking...
[codecarbon INFO @ 12:32:32] [setup] CPU Tracking...
[codecarbon WARNING @ 12:32:32] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon INFO @ 12:32:33] CPU Model on constant consumption mode: AMD EPYC 7742 64-Core Processor
[codecarbon INFO @ 12:32:33] [setup] GPU Tracking...
[codecarbon INFO @ 12:32:33] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 12:32:33] >>> Tracker's metadata:
[codecarbon INFO @ 12:32:33]   Platform system: Linux-5.15.0-113-generic-x86_64-with-glibc2.31
[codecarbon INFO @ 12:32:33]   Python version: 3.10.14
[codecarbon INFO @ 12:32:33]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 12:32:33]   Available RAM : 503.532 GB
[codecarbon INFO @ 12:32:33]   CPU count: 128
[codecarbon INFO @ 12:32:33]   CPU model: AMD EPYC 7742 64-Core Processor
[codecarbon INFO @ 12:32:33]   GPU count: 4
[

OutOfMemoryError: CUDA out of memory. Tried to allocate 44.00 MiB. GPU 0 has a total capacity of 39.39 GiB of which 2.19 MiB is free. Process 2865496 has 26.51 GiB memory in use. Process 2868301 has 1.31 GiB memory in use. Process 2871299 has 3.91 GiB memory in use. Including non-PyTorch memory, this process has 7.63 GiB memory in use. Of the allocated memory 7.07 GiB is allocated by PyTorch, and 59.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

[codecarbon INFO @ 12:32:37] Energy consumed for all GPUs : 0.000224 kWh. Total GPU Power : 714.2588072720782 W
[codecarbon INFO @ 12:32:37] 0.000313 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:32:38] Energy consumed for RAM : 0.000099 kWh. RAM Power : 188.8243260383606 W
[codecarbon INFO @ 12:32:38] Energy consumed for all CPUs : 0.000062 kWh. Total CPU Power : 112.5 W
[codecarbon INFO @ 12:32:38] Energy consumed for all GPUs : 0.000385 kWh. Total GPU Power : 666.8024904979694 W
[codecarbon INFO @ 12:32:38] 0.000546 kWh of electricity used since the beginning.
[codecarbon INFO @ 12:32:39] Energy consumed for RAM : 0.000151 kWh. RAM Power : 188.8243260383606 W
[codecarbon INFO @ 12:32:39] Energy consumed for all CPUs : 0.000093 kWh. Total CPU Power : 112.5 W
[codecarbon INFO @ 12:32:39] Energy consumed for all GPUs : 0.000515 kWh. Total GPU Power : 477.60685573345586 W
[codecarbon INFO @ 12:32:39] 0.000758 kWh of electricity used since the beginning.
[codecarbon 